In [8]:
''' 
Part1:

1. Create a python function to generate data : Sample data and store in a CSV file in you laptop.
The file name: order_data_20220301.csv
orderid,brand_name,product_name,sales_ammount,sales_date

brand_name:['Apple','Samsung','Nokia']
product_name is Key value pair
product_name:{'Apple':['iphon11','iphone12','iphone13','iphoneSE','IpadMax','IpadMini','laptop256','Macbook512'],
'Samsung':['galaxy10','galaxy11','galaxy12','galaxy13','watch320','watch340'],
'Nokia':['Nk320',''Nk400',''Nk500']
orderid: Unique id with 7 digit number

Finally data should be like this
1002346,Apple,iphon11,450,2022-01-06
1002246,Apple,iphon11,460,2022-02-01
1002546,Apple,iphoneSE,350,2022-03-06

'''

import pandas as pd
import random
import datetime


def random_date(start : datetime, end : datetime):
    diff = datetime.timedelta(days=(end - start).days)
    rae = random.random() * diff
    return start + datetime.timedelta(days = rae.days)


def generate_dummy_info(record_count):
    print(record_count)
    ran_data = []
    start_date = datetime.datetime(2018, 5, 1)
    end_date = datetime.datetime(2020, 5, 1)

    for i in range(0, record_count):
        id_start = f"{i+1:07}"
        brand=random.choice(['Apple','Samsung','Nokia'])
        if brand=='Apple':
            product=random.choice(['iphon11','iphone12','iphone13','iphoneSE','IpadMax','IpadMini','laptop256','Macbook512'])
        elif brand=='Samsung':
            product=random.choice(['galaxy10','galaxy11','galaxy12','galaxy13','watch320','watch340'])
        else:
            product=random.choice(['Nk320','Nk400','Nk500'])
        sales_ammt = (random.randint(0, 9999999))/100
        sales_date = random_date(start_date,end_date)
        ran_data.append((id_start, brand, product, sales_ammt, sales_date))

    return ran_data



record_num_request = 50000
df = pd.DataFrame(generate_dummy_info(record_num_request), columns=['orderid', 'brand_name', 'product_name', 'sales_ammount', 'sales_date'])
df.to_csv('order_data_20230401.csv', index= False)

50000


In [9]:
'''
1. Build redshift architecture diagram.
SEE /redshift/project/redshift_diagram.jpg

2. What is columnar data format? what are the advantages?
Data is stored on the disk in column order opposed to row order. 
Storing in this way enables:
    1. reduces amount of data you need to load by using disk space more efficiently. If the block size is smaller than a single record, the space is wasted -- avoided with columnar data storage
    2. since things are stored in column order, only the selected columns are loaded instead of loading each row and then parsing them 
    3. compression for each data type which improves read/write IO by minimizing the amount of disk space used therefore minimizing the CPU usage and time
        - also network transfer through nodes is reduced as well
  

3. What is encoding? Different type of encoding? A sample create table statement with encoding.

Encoding in AWS is used in compression / decompression of data to reduce the amount of disk space used.

SEE redshift_data_encoding.png

CREATE TABLE IF NOT EXISTS cards_ingest.tran_fact(
    tran_id int encode delta,
    cust_id varchar(10) encode raw, 
    stat_cd varchar(2) encode raw, 
    tran_ammt decimal(10,2) encode AZ64,
    tran_date date encode delta
) SORTKEY(tran_date);



4. What is distribution key? How it is helpful in query performance.
A distribution key is a column(s) that is used to determine what node data is stored on in Redshift. 
If a lot of data is stored on the same node, this reduces performance because you are not sharing the processing load.
So depending on the problem you can 

ALL key -- makes table stored on every node
    if joining / GROUP BY on a certain column frequently, and there are rows on seperate clusters, the nodes need to send over the data over the network to another node so that it can do the computation. 
Even key -- makes table evenly distributed throughout nodes, to be able to share processing load

Sort key -- stores data in memory in sorted order.
    makes various queries more efficient such as MAX, MIN, ORDER BY
    however it increases insert times



'''


'\n1. Build redshift architecture diagram.\nSEE /redshift/project/redshift_diagram.jpg\n\n2. What is columnar data format? what are the advantages?\nData is stored on the disk in column order opposed to row order. \nStoring in this way enables:\n    1. reduces amount of data you need to load by using disk space more efficiently. If the block size is smaller than a single record, the space is wasted -- avoided with columnar data storage\n    2. since things are stored in column order, only the selected columns are loaded instead of loading each row and then parsing them \n    3. compression for each data type which improves read/write IO by minimizing the amount of disk space used therefore minimizing the CPU usage and time\n        - also network transfer through nodes is reduced as well\n  \n\n3. What is encoding? Different type of encoding? A sample create table statement with encoding.\n\nEncoding in AWS is used in compression / decompression of data to reduce the amount of disk spa